In [45]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

img_dir = "/kaggle/input/fingerprint-dataset-for-fvc2000-db4-b/dataset_FVC2000_DB4_B/dataset/train_data"
images = []
labels = []

for filename in os.listdir(img_dir):
    if filename.endswith(".bmp") or filename.endswith(".png") or filename.endswith(".tif"):
        img_path = os.path.join(img_dir, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        
        img = cv2.resize(img, (128, 128))
        images.append(img)

        
        label = filename.split("_")[0]  
        labels.append(label)
        






In [46]:
from skimage.feature import hog
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Assuming images and labels are already defined
hog_features = []

# Maximum length of the features for consistent shape
max_feature_length = 0

# Extract HOG features from each image
for image in images:
    features = hog(image, orientations=9, pixels_per_cell=(8, 8),
                   cells_per_block=(2, 2), block_norm='L2-Hys')

    

    # Update max_feature_length
    max_feature_length = max(max_feature_length, len(features))

    # Append the filtered features
    hog_features.append(features)

# Pad features to ensure they all have the same length
for i in range(len(hog_features)):
    feature_len = len(hog_features[i])
    if feature_len < max_feature_length:
        # Pad with zeros to match the maximum length
        hog_features[i] = np.pad(hog_features[i], (0, max_feature_length - feature_len), 'constant')

# Convert the features to a numpy array for SVM
X = np.array(hog_features)

# Encode labels
le = LabelEncoder()
y = le.fit_transform(labels)





In [47]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Encoder les labels texte
le = LabelEncoder()
y = le.fit_transform(labels)

# Séparation en train/test
X_train, X_test, y_train, y_test = train_test_split(hog_features, y, test_size=0.2, random_state=42)

# Créer et entraîner le SVM avec le noyau polynomial
clf = SVC(kernel='poly')  # Par défaut, degré=3
clf.fit(X_train, y_train)

# Évaluer la précision
accuracy = clf.score(X_test, y_test)
print(f"Accuracy with polynomial kernel: {accuracy}")




Accuracy with polynomial kernel: 0.85625
